In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

/Users/harshitgupta/Desktop/vs /VS-Data-Den/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
data = pd.read_csv('/Users/harshitgupta/Desktop/vs /VS-Data-Den/rm_dataset.csv')


In [4]:
data['date_field'] = pd.to_datetime(data['date_field'])


data['regional_master'] = data['regional_master'].astype('category')

In [5]:
data.sort_values(by='date_field', inplace=True)

In [6]:
look_back = 3


epochs = 100
batch_size = 32

In [7]:
def prepare_data(df):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(df[['sales']])
    
 
    X, y = [], []
    for i in range(len(df)):
        if i + look_back < len(df):  # Add this condition to avoid out-of-bounds error
            X.append(scaled_data[i:i+look_back])
            y.append(scaled_data[i + look_back, 0])
        else:
            print(f"Index {i} is out of bounds for axis 0 with size {len(df)}")
    return np.array(X), np.array(y), scaler


In [8]:
def create_model():
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(look_back, 1)))
    model.add(LSTM(units=50))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [9]:
predictions = {}
actual_values = {}

In [10]:
for region in data['regional_master'].unique():
    region_data = data[data['regional_master'] == region].reset_index(drop=True)
    actual_values[region] = region_data['sales'].values[-12:]  # Actual sales for last 12 weeks
    
    X, y, scaler = prepare_data(region_data)
    
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))

    model = create_model()
    model.fit(X, y, epochs=epochs, batch_size=batch_size, verbose=0)
    

    last_sequence = X[-1]
    forecast = []
    for i in range(12):  # Predicting for 12 weeks (adjust as needed)
        prediction = model.predict(np.array([last_sequence]))[0][0]
        forecast.append(prediction)
        last_sequence = np.append(last_sequence[1:], [[prediction]], axis=0)

    forecast = scaler.inverse_transform(np.array(forecast).reshape(-1, 1))
    predictions[region] = forecast.flatten()


Index 322 is out of bounds for axis 0 with size 325
Index 323 is out of bounds for axis 0 with size 325
Index 324 is out of bounds for axis 0 with size 325


2024-04-25 13:30:18.152638: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-04-25 13:30:18.152657: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-04-25 13:30:18.152662: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-04-25 13:30:18.152679: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-04-25 13:30:18.152689: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
/Users/harshitgupta/Desktop/vs /VS-Data-Den/.venv/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. W

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Index 322 is out of bounds for axis 0 with size 325
Index 323 is out of bounds for axis 0 with size 325
Index 324 is out of bounds for axis 0 with size 325


/Users/harshitgupta/Desktop/vs /VS-Data-Den/.venv/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
Index 323 is out of bounds for axis 0 with size 326
Index 324 is out of bounds for axis 0 with size 326
Index 325 is out of bounds for axis 0 with size 326


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 284ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Index 89 is out of bounds for axis 0 with size 92
Index 90 is out of bounds for axis 0 with size 92
Index 91 is out of bounds for axis 0 with size 92


/Users/harshitgupta/Desktop/vs /VS-Data-Den/.venv/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 275ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Index 16 is out of bounds for axis 0 with size 19
Index 17 is out of bounds for axis 0 with size 19
Index 18 is out of bounds for axis 0 with size 19


/Users/harshitgupta/Desktop/vs /VS-Data-Den/.venv/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 285ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step


In [12]:
mape_values = {}

In [13]:
for region in predictions:
    actual = actual_values[region]
    predicted = predictions[region]
    mape = np.mean(np.abs((actual - predicted) / actual)) * 100
    mape_values[region] = mape

/var/folders/_p/ljb0l43112vc95swz8nhgv340000gn/T/ipykernel_61571/4278013097.py:4: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((actual - predicted) / actual)) * 100


In [21]:
overall_actual = np.array([])
overall_predicted = np.array([])
for region, forecast in predictions.items():
    region_data = data[data['regional_master'] == region].reset_index(drop=True)
    actual = region_data.iloc[-12:, :]['sales']
    overall_actual = np.concatenate((overall_actual, actual.values))
    overall_predicted = np.concatenate((overall_predicted, forecast))

In [23]:
from sklearn.metrics import mean_absolute_percentage_error

In [24]:
overall_mape = mean_absolute_percentage_error(overall_actual, overall_predicted)
print(f'Overall MAPE: {overall_mape}')

Overall MAPE: 7.255047070178214e+19
